# Base de datos neo4j

## Lectura de datos

In [1]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

## Conexión con neo4j y borrado de BB.DD.

In [2]:
%load_ext cypher

In [3]:
from py2neo import Graph, Node, Relationship

graph = Graph()

# Borrado de todos los nodos y relaciones
graph.delete_all()

## Carga de datos en neo4j

 El modelo de nodos y relaciones que voy a considerar es este:
 
 ![Nodos y relaciones neo4j](./img/nodosRelacionesNEO.jpg)
 
 
 

Desnormalizo los datos de ascensos para facilitar la introducción de nodos y relaciones en la base de datos

In [4]:
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = pd.merge(df_ascensos, df_dificultades, on = ['id_dificultad'], how = 'inner');
df_ascensos = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');

# Renombrar campos por claridad (el merge auto-asigna nombres a campos repetidos)
df_ascensos = df_ascensos.rename(
    columns={
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});

In [5]:
df_ascensos.head(2)

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais_risco,tipo_encadenamiento,grado_frances,grado_usa,grado_bloque_usa,nombre_escalador,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,anio_comienzo
0,20384,33,2,LE DE,FONTAINEBLEAU,ROCHER CANON,2017-04-17,FRA,Flash,5c,5.9,V2,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003
1,20384,33,2,SCHMALSEITE DV,ELBSANDSTEIN,KRALLENTURM,2017-06-10,DEU,Flash,5c,5.9,V2,Clemens Kurth,Hombre,1982-04-24,Alfeld (Leine),DEU,2003


Convierto el data frame de ascensos desnormalizado a JSON para la inserción de información en neo4j

In [6]:
import json
json_string = df_ascensos.to_json(orient = 'records')
json_list = json.loads(json_string)

Iterando en el data frame de ascensos, voy añadiendo los nodos y las relaciones que se generan por cada ascenso:

In [7]:
for index, ascenso_json in enumerate(json_list):
    # Nodos
    escalador = Node("Escalador", nombre = ascenso_json["nombre_escalador"], anio_comienzo = ascenso_json["anio_comienzo"]) 
    via = Node("Via", nombre=ascenso_json["nombre_via"])
    dificultad = Node("Dificultad", valor = ascenso_json["id_dificultad"], grado_frances = ascenso_json["grado_frances"])
    risco = Node("Risco", nombre = ascenso_json["risco"])
    pais = Node("Pais", id=ascenso_json["pais_risco"])
    
  
        
    # Relaciones  
    rel_escalador_via = Relationship(escalador, "ASCIENDE", via, tipo_encadenamiento = ascenso_json["tipo_encadenamiento"])
    rel_escalador_pais = Relationship(escalador, "NACE", pais)
    rel_via_dificultad = Relationship(via, "POSEE", dificultad)
    rel_via_risco = Relationship(via, "PERTENECE", risco)   
    rel_risco_pais = Relationship(risco, "SE UBICA EN", pais)
    
    graph.merge(escalador|via|dificultad|risco|pais|rel_escalador_pais|rel_via_dificultad|rel_via_risco|rel_risco_pais)
    graph.create(rel_escalador_via)
    
    if index % 500 == 0:
        print index

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500


Además, como voy a tener que hacer calculos de dificultad media, necesito que los todos los nodos dificultad existan, aunque no hayan existido ascensos con esa dificultad. Por ello, los añado siguiendo una estrategia de unicidad (merge) iterando sobre el data frame de dificultades:

In [8]:
import json
json_string = df_dificultades.to_json(orient = 'records')
json_list = json.loads(json_string)

for index, dificultad_json in enumerate(json_list):
    dificultad = Node("Dificultad", valor = dificultad_json["id_dificultad"],
                                    grado_frances = dificultad_json["grado_frances"])
    graph.merge(dificultad)
    
    if index % 10 == 0:
        print index

0
10
20
30
40
50
60
70
80


In [19]:
df_ascensos['nombre_via'].value_counts()

NO NAME                          13
CROSTA PANIC                      8
MOBY DICK                         7
RED BULL                          7
NN                                6
PANIC ROOM                        6
UNKNOWN                           5
THE FIN                           5
-                                 5
KIEDY KOBIETA PłACZE              5
ANDROMEDA                         5
ASTERIX                           5
GOLDEN BOY                        5
BEATLE JUICE                      5
BOSNA GENIAL                      5
PRIMAVERA                         5
FIRST                             5
THE RHINO                         5
ECLIPSE                           5
EMANUELLE                         4
MAGNETOWID                        4
THEY CALL HIM JORDAN              4
NORMALWEG                         4
ZARZAMORA                         4
ANDRZEJ LEPPER                    4
CARPE DIEM                        4
UNDERGROUND                       4
CARNAGE                     

In [28]:
df_ascensos[df_ascensos['nombre_via']=='RED BULL']

,id_escalador,id_dificultad,id_tipo_encadenamiento,nombre_via,risco,sector,fecha,pais_risco,tipo_encadenamiento,grado_frances,grado_usa,grado_bloque_usa,nombre_escalador,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,anio_comienzo
5996,28151,55,1,RED BULL,HOHE WAND,HOCHKOGEL,2017-03-01,AUT,Redpoint,7b+,5.12c,V8/9,michael moser,Hombre,1982-10-30,vienna,AUT,2007
6629,4650,44,3,RED BULL,YANGSHUO,THE THUMB,2017-06-08,CHN,Onsight,6c,5.11a,V5,Tj Ovesen,Hombre,1971-10-30,---,NOR,1998
6803,45182,55,1,RED BULL,HOHE WAND,HOCHKOGEL,2017-02-23,AUT,Redpoint,7b+,5.12c,V8/9,Andreas Moser,Hombre,1985-07-01,Vienna,AUT,0
8728,44871,53,1,RED BULL,MOZET,ARENDSMASSIEF,2017-08-13,BEL,Redpoint,7b,5.12b,V8,Dennis Muys,Hombre,1988-06-30,Brecht,BEL,2004
9512,66925,59,1,RED BULL,OLIANA,OLIANA,2017-04-01,ESP,Redpoint,7c+,5.13a,V10,Chaken Gómez conde,Hombre,2555-12-31,Badalona,ESP,0
11143,49021,59,3,RED BULL,TERRADETS,BRUIXES,2017-02-09,ESP,Onsight,7c+,5.13a,V10,Maciej Dobrzański,Hombre,2555-12-31,KrK,POL,0
11312,2604,64,2,RED BULL,OLIANA,CONTRAFORT DE RUMBAU,2017-02-10,ESP,Flash,8a+,5.13c,V12,Pierre Délas,Hombre,1980-12-05,Massy,FRA,1991
